# Ingest ISO 3166 data into Trino pipeline

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

Run these in a notebook cell if you need to install onto your nb env

```python
# 'capture' magic prevents long outputs from spamming your notebook
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
```

In [1]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

import trino
from sqlalchemy.engine import create_engine
from sqlalchemy import text

The python module `pycountry` has all the data we need, and we can use it within python/pandas.

We import the data here so we can use it in SQL/Trino queries.

In [2]:
import pycountry

Enter the Pandas!

In [3]:
import pandas as pd
import numpy as np
import io

Load Environment Variables

In [4]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

In [5]:
trino_bucket = osc.attach_s3_bucket("S3_DEV")

Open a Trino connection using JWT for authentication

In [6]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'mdt_sandbox'

engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog, schema=ingest_schema)

using connect string: trino://MichaelTiemannOSC@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev/mdt_sandbox


In [7]:
# Show available schemas to ensure trino connection is set correctly
schema_read = osc._do_sql(f'show schemas in {ingest_catalog}', engine, verbose=True)
for row in schema_read:
    print(row)

show schemas in osc_datacommons_dev
[('aicoe_osc_demo_results',), ('default',), ('demo_dv',), ('dera',), ('essd',), ('iceberg_demo',), ('information_schema',), ('ingest',), ('mdt_sandbox',), ('pcaf_sovereign_footprint',), ('rmi',), ('sandbox',), ('wri_gppd',)]
('aicoe_osc_demo_results',)
('default',)
('demo_dv',)
('dera',)
('essd',)
('iceberg_demo',)
('information_schema',)
('ingest',)
('mdt_sandbox',)
('pcaf_sovereign_footprint',)
('rmi',)
('sandbox',)
('wri_gppd',)


Drop previous tables and schema to start with a fresh slate

In [8]:
for ingest_table in [ 'countries', 'subdivisions' ]:
    sql = f"""
drop table if exists {ingest_schema}.{ingest_table}
"""
    qres = osc._do_sql(sql, engine, verbose=True)


drop table if exists mdt_sandbox.countries


drop table if exists mdt_sandbox.subdivisions



In [9]:
sql = f"show tables in {ingest_schema}"
qres = osc._do_sql(sql, engine, verbose=True)
print(qres)

show tables in mdt_sandbox
[('isic_to_sector',), ('itr_portfolio_universe',), ('pcaf_dbt_models',), ('portfolio_universe',), ('saved_queries',), ('sf_oecd_exch_rates',), ('sf_oecd_exch_rates_source',), ('sf_oecd_exgr_dco2',), ('sf_oecd_exgr_dco2_source',), ('sf_oecd_imgr_fco2',), ('sf_oecd_imgr_fco2_source',), ('sf_primap_hist_emissions',), ('sf_primap_hist_emissions_source',), ('sf_total_sovereign_emissions',), ('sf_total_sovereign_emissions_source',), ('sf_unfccc_countries_source',), ('sf_unfccc_results',), ('sf_unfccc_results_source',), ('sf_unfccc_with_lulucf',), ('sf_unfccc_with_lulucf_source',), ('sf_unfccc_without_lulucf',), ('sf_unfccc_without_lulucf_source',), ('sf_wdi_gdp',), ('sf_wdi_gdp_source',), ('sf_wdi_population',), ('sf_wdi_population_source',)]
[('isic_to_sector',), ('itr_portfolio_universe',), ('pcaf_dbt_models',), ('portfolio_universe',), ('saved_queries',), ('sf_oecd_exch_rates',), ('sf_oecd_exch_rates_source',), ('sf_oecd_exgr_dco2',), ('sf_oecd_exgr_dco2_source'

One table we want to link up with is defined in the ESSD schema.  It links ISO Alpha3 codes with UN regions.

In [10]:
with engine.begin() as cxn:
    qres = cxn.execute(text("select * from essd.regions")).fetchall()[0:5]

In [11]:
df = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df.replace({np.NaN:pd.NaT}, inplace=True)
df_countries = df.convert_dtypes()
df_countries

,alpha_2,alpha_3,flag,name,numeric,official_name,common_name
0,AW,ABW,🇦🇼,Aruba,533,NaT,NaN
1,AF,AFG,🇦🇫,Afghanistan,004,Islamic Republic of Afghanistan,NaN
2,AO,AGO,🇦🇴,Angola,024,Republic of Angola,NaN
3,AI,AIA,🇦🇮,Anguilla,660,NaT,NaN
4,AX,ALA,🇦🇽,Åland Islands,248,NaT,NaN
...,...,...,...,...,...,...,...
244,WS,WSM,🇼🇸,Samoa,882,Independent State of Samoa,NaN
245,YE,YEM,🇾🇪,Yemen,887,Republic of Yemen,NaN
246,ZA,ZAF,🇿🇦,South Africa,710,Republic of South Africa,NaN
247,ZM,ZMB,🇿🇲,Zambia,894,Republic of Zambia,NaN


In [12]:
df = pd.DataFrame([subdivision.__dict__['_fields'] for subdivision in pycountry.subdivisions])
display(df[df.parent_code.notna()])
df.replace({np.NaN:pd.NaT}, inplace=True)
df_subdivisions = df.convert_dtypes()
df_subdivisions

,code,name,type,parent_code,country_code,parent
146,AZ-BAB,Babək,Rayon,AZ-NX,AZ,NX
153,AZ-CUL,Culfa,Rayon,AZ-NX,AZ,NX
165,AZ-KAN,Kǝngǝrli,Rayon,AZ-NX,AZ,NX
175,AZ-NV,Naxçıvan,Municipality,AZ-NX,AZ,NX
178,AZ-ORD,Ordubad,Rayon,AZ-NX,AZ,NX
...,...,...,...,...,...,...
4850,UG-431,Rukiga,District,UG-W,UG,W
4851,UG-432,Kikuube,District,UG-W,UG,W
4852,UG-433,Kazo,District,UG-W,UG,W
4853,UG-434,Kitagwenda,District,UG-W,UG,W


,code,name,type,parent_code,country_code,parent
0,AD-02,Canillo,Parish,NaN,AD,NaN
1,AD-03,Encamp,Parish,NaN,AD,NaN
2,AD-04,La Massana,Parish,NaN,AD,NaN
3,AD-05,Ordino,Parish,NaN,AD,NaN
4,AD-06,Sant Julià de Lòria,Parish,NaN,AD,NaN
...,...,...,...,...,...,...
5118,ZW-MI,Midlands,Province,NaN,ZW,NaN
5119,ZW-MN,Matabeleland North,Province,NaN,ZW,NaN
5120,ZW-MS,Matabeleland South,Province,NaN,ZW,NaN
5121,ZW-MV,Masvingo,Province,NaN,ZW,NaN


In [13]:
for ingest_table, df in [ ('countries',df_countries), ('subdivisions', df_subdivisions)]:
    drop_table = osc._do_sql(f"drop table if exists {ingest_schema}.{ingest_table}", engine, verbose=True)
    df.to_sql(ingest_table, con=engine, 
              schema=ingest_schema, if_exists='append',
              index=False,
              method=osc.TrinoBatchInsert(batch_size = 2000, verbose = True))
    dataset_query = (f'SELECT * FROM {ingest_schema}.{ingest_table} limit 10')
    dataset = osc._do_sql(dataset_query, engine, verbose=True)
    for row in dataset:
        print(row)

drop table if exists mdt_sandbox.countries
constructed fully qualified table name as: "mdt_sandbox.countries"
inserting 249 records
  ('AW', 'ABW', '🇦🇼', 'Aruba', '533', NULL, NULL)
  ('AF', 'AFG', '🇦🇫', 'Afghanistan', '004', 'Islamic Republic of Afghanistan', NULL)
  ('AO', 'AGO', '🇦🇴', 'Angola', '024', 'Republic of Angola', NULL)
  ...
  ('ZW', 'ZWE', '🇿🇼', 'Zimbabwe', '716', 'Republic of Zimbabwe', NULL)
batch insert result: [(249,)]
SELECT * FROM mdt_sandbox.countries limit 10
[('AW', 'ABW', '🇦🇼', 'Aruba', '533', None, None), ('AF', 'AFG', '🇦🇫', 'Afghanistan', '004', 'Islamic Republic of Afghanistan', None), ('AO', 'AGO', '🇦🇴', 'Angola', '024', 'Republic of Angola', None), ('AI', 'AIA', '🇦🇮', 'Anguilla', '660', None, None), ('AX', 'ALA', '🇦🇽', 'Åland Islands', '248', None, None), ('AL', 'ALB', '🇦🇱', 'Albania', '008', 'Republic of Albania', None), ('AD', 'AND', '🇦🇩', 'Andorra', '020', 'Principality of Andorra', None), ('AE', 'ARE', '🇦🇪', 'United Arab Emirates', '784', None, None), (